# 5.3 PyTorch神经网络工具箱

PyTorch中的nn模块是专门为神经网络设计的模块化接口工具箱。

## 5.3.1 一维卷积类nn.Conv1d

一维卷积常用于文本等序列数据，只对宽度进行卷积，对高度不卷积。例如，含有L各词（或字）的文本，可认为其长度为L，每个词或字的特征（或称为嵌入向量）大小为D，则文本的特征可表示为形状为(D，L)的张量，卷积核窗口在句子长度的方向上滑动，即可进行一维卷积操作。

下面给出一维卷积的示例：

In [3]:
import torch
from torch import nn
conv1 = nn.Conv1d(in_channels=256, out_channels=100, kernel_size=3, stride=1, padding=0)  # 定义一个一维卷积实例
input = torch.randn(32, 35, 256)  # 定义输入特征图张量，形状为[batch_size, L, D],参数分别为批大小batch_size、最大长度L和特征维度D
input = input.permute(0, 2, 1)  # 张量交换维度，形状变为[batch_size, D, L]
out = conv1(input)  #进行一维卷积操作，输出特征图张量形状为[batch_size, out_channels, (L + 2 * padding - kernel_size) / stride + 1]
print(out.shape)  # 打印输出张量的形状

torch.Size([32, 100, 33])


分析以上代码已知，input张量的形状是[batch_size, L, D]，在开始Conv1d前需要将L换到最后一个维度，卷积核在该维度上滑动进行一维卷积。卷积后的结果的形状为[batch_size, out_channels, (L + 2 * padding - kernel_size) / stride + 1]，在纵向维度上的值为(35 + 2*0 - 3)/1 + 1=33，即卷积后序列的特征图长度由原来的35变为了33。

一维卷积中含有要学习的参数，其中权值参数个数为in_channels * out_channels * kernel_size，若带有偏置项，则偏置项个数为out_channels。上例中，可以用如下代码打印所有参数的当前值：

In [4]:
print(list(conv1.parameters()))

[Parameter containing:
tensor([[[-0.0217, -0.0190, -0.0277],
         [-0.0032, -0.0023, -0.0356],
         [-0.0073,  0.0023, -0.0221],
         ...,
         [ 0.0064, -0.0235, -0.0195],
         [-0.0043, -0.0019, -0.0160],
         [ 0.0285,  0.0345,  0.0334]],

        [[ 0.0205,  0.0354, -0.0247],
         [ 0.0121,  0.0197, -0.0333],
         [ 0.0223, -0.0299,  0.0344],
         ...,
         [ 0.0075, -0.0298,  0.0130],
         [ 0.0001,  0.0258, -0.0264],
         [-0.0227,  0.0062, -0.0156]],

        [[-0.0059, -0.0349,  0.0256],
         [-0.0280, -0.0288, -0.0291],
         [-0.0194, -0.0004,  0.0214],
         ...,
         [-0.0278,  0.0264,  0.0353],
         [ 0.0285, -0.0163, -0.0147],
         [ 0.0028,  0.0240, -0.0045]],

        ...,

        [[ 0.0305, -0.0228, -0.0079],
         [ 0.0208, -0.0116, -0.0079],
         [-0.0034, -0.0167, -0.0026],
         ...,
         [ 0.0253, -0.0160,  0.0176],
         [ 0.0074, -0.0003,  0.0332],
         [ 0.0270,  0.0352,

## 5.3.2 二维卷积类nn.Conv2d

二维卷积常用于图像数据，同时对宽和高进行卷积处理。对于输入为(channel, height, width)的图像，其中channel为图像的通道数，height为图像的高度，width为图像的宽度，卷积核从左到右，从上到下对图像进行卷积操作。

假设输入张量数据形状为[10, 16, 64, 64]，表示batch_size为10，即一次输入10张二维矩阵数据，每组数据的通道数为16，矩阵的宽度和高度分别为64和64。设卷积核大小为(16,3,3)，则进行二维卷积的示例如下：

In [5]:
x = torch.randn(10, 16, 64, 64)  # 参数分别为batch_size, channel, height, width
m = nn.Conv2d(16, 32, (3, 3), (1, 1))  # in_channel, out_channel ,kennel_size,stride
y = m(x)
print(y.shape) 

torch.Size([10, 32, 62, 62])


二维卷积对宽和高的卷积结果形状都为[batch_size, out_channels, (L + 2 * padding - kernel_size) / stride + 1]，其中L为矩阵数据的宽或高，stride都为1，padding默认为0，所以输出张量横向和纵向的维度均为(64+2*0-3)/1 + 1 = 62。

二维卷积中也含有要学习的参数，其中权值参数个数为in_channels * out_channels * kernel_size * kernel_size，若带有偏置项，则偏置项个数为out_channels。和观察一维卷积权值参数类似，上例中也可以用如下代码打印所有参数的当前值：


In [6]:
print(list(m.parameters()))

[Parameter containing:
tensor([[[[ 0.0801, -0.0273, -0.0643],
          [ 0.0789,  0.0441, -0.0523],
          [ 0.0166,  0.0822, -0.0010]],

         [[ 0.0655,  0.0538,  0.0600],
          [ 0.0034,  0.0211, -0.0735],
          [-0.0267,  0.0601, -0.0425]],

         [[ 0.0259, -0.0742, -0.0286],
          [ 0.0156,  0.0514,  0.0310],
          [-0.0250, -0.0490,  0.0581]],

         ...,

         [[-0.0803,  0.0046,  0.0509],
          [ 0.0830, -0.0300, -0.0612],
          [ 0.0380,  0.0217,  0.0215]],

         [[-0.0197, -0.0788,  0.0693],
          [ 0.0647,  0.0369, -0.0811],
          [ 0.0130, -0.0064,  0.0568]],

         [[-0.0365, -0.0166,  0.0423],
          [-0.0661, -0.0087,  0.0152],
          [ 0.0707, -0.0078, -0.0584]]],


        [[[ 0.0002,  0.0160, -0.0259],
          [-0.0423, -0.0599,  0.0428],
          [-0.0657,  0.0787,  0.0359]],

         [[ 0.0262,  0.0137,  0.0631],
          [-0.0128, -0.0151, -0.0154],
          [ 0.0109, -0.0255, -0.0722]],

        

## 5.3.3 全连接类nn.Linear

全连接类作用于设置网络中的全连接层，对输入数据进行线性转换，并存储权重和偏置。通常，全连接操作的输入与输出都是二维张量，一般形状为[batch_size, size]，不同于二维卷积要求输入和输出的是四维张量。

参数in_features表示输入维度的大小，out_features表示输出维度的大小，bias表示是否带偏置，默认是带偏置的。示例代码如下：

In [7]:
connected_layer = nn.Linear(in_features = 64*64*3, out_features = 1)  # 输入维度为64*64*3，输出维度为1
input = torch.randn(10, 3, 64, 64)
input = input.view(10, 64*64*3)  #torch.Size([10, 12288])
output = connected_layer(input) # 调用全连接层
print(output.shape)

torch.Size([10, 1])


上例中，input调用view函数完成了对张量形状的调整，由原来的(10, 3, 64, 64)调整为(10, 12288)。在这里，也可以使用reshape()方法对张量形状进行调整，例如：

In [8]:
input = input.reshape((10, 64*64*3))

全连接层中也含有要学习的参数，其中权值参数个数为in_features * out_features，若带有偏置项，则偏置项个数为out_features。和前面类似，上例中也可以用如下代码打印所有参数的当前值：

In [9]:
print(list(connected_layer.parameters()))

[Parameter containing:
tensor([[ 0.0057,  0.0075,  0.0081,  ..., -0.0037, -0.0030,  0.0077]],
       requires_grad=True), Parameter containing:
tensor([0.0076], requires_grad=True)]


## 5.3.4 平坦化类nn.Flatten

上一节的示例中，input = input.view(10, 64*64*3)语句实现了对张量形状的调整。实际上，nn模块还提供了Flatten类，也可以直接把指定的连续几维数据展平为连续的一维数据，默认从第1维到最后一维进行平坦化，第0维常表示batch_size，因此不进行展平。

由此，上例中的input = input.view(10, 64*64*3)语句，也可以直接写成：

In [10]:
Flatten = nn.Flatten()  # 实例化类对象
input = Flatten(input)  # 进行展平，input的shape也是torch.Size([10, 12288])

可见，两者的效果是一样的。但nn.Flatten作为一种操作，可以放到顺序化容器（nn.Sequential）中，更具有通用性。

## 5.3.5 非线性激活函数

PyTorch的nn模块提供了丰富的非线性激活函数，用来对模型的输入和输出构建复杂的映射。例如ReLU、Softmax、Sigmoid、Tanh、LogSigmoid、LogSoftmax等。激活函数常用于在线性变换后，通过加入非线性变换使得模型能进行更复杂的表示。

下面以nn.ReLU为例，介绍激活函数的使用。ReLU类定义如下：

In [11]:
nn.ReLU(inplace = False)

ReLU()

下面的代码是在线性层后引入非线性层的示例：

In [12]:
input = torch.randn(4, 3, 64, 64)
Flatten = nn.Flatten()  # 实例化类对象
flat_image = Flatten(input)  # 进行展平，input的shape也是torch.Size([10, 12288])
layer1 = nn.Linear(in_features=64*64*3, out_features=5)
hidden1 = layer1(flat_image)
print(hidden1.size())
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

torch.Size([4, 5])
Before ReLU: tensor([[ 0.4898, -0.8552,  0.3207, -0.6352,  1.1304],
        [ 0.0442,  0.4436,  0.1707,  0.6401,  0.2983],
        [ 0.3294,  1.2857, -0.2988, -0.1367, -1.0043],
        [ 0.5604, -0.0085,  0.1123, -1.1393,  0.4354]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.4898, 0.0000, 0.3207, 0.0000, 1.1304],
        [0.0442, 0.4436, 0.1707, 0.6401, 0.2983],
        [0.3294, 1.2857, 0.0000, 0.0000, 0.0000],
        [0.5604, 0.0000, 0.1123, 0.0000, 0.4354]], grad_fn=<ReluBackward0>)


不难发现，经过ReLU函数后，所有的负值都变为了0。其他激活函数使用方法类似，将输入数据进行非线性处理。通常，激活函数都没有要学习的参数。

## 5.3.6 顺序化容器nn.Sequential

顺序化容器可以将神经网络的各个模块，如卷积操作、全连接操作、激活函数等，按照顺序加入其中，模型在训练或推理时，按顺序执行各个操作。除此之外，也可以将多个模块放在有序字典里面进行传递。
下面给出几种顺序化容器的实例化方法。

（1）定义时直接加入模块

示例如下：

In [13]:
input = torch.randn(4, 3, 64, 64)
net = nn.Sequential(
    nn.Conv2d(3, 32, (3, 3), (1, 1)),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(32*62*62, 2),
)
output = net(input)
print(f"output: {output}")  # 打印输出
print(net)  # 打印网络模型结构

output: tensor([[-0.0147,  0.1290],
        [ 0.0379,  0.0206],
        [ 0.0873,  0.2238],
        [ 0.1602,  0.0237]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Flatten(start_dim=1, end_dim=-1)
  (3): Linear(in_features=123008, out_features=2, bias=True)
)


（2）先定义对象，后加入模块

示例如下：

In [14]:
net = nn.Sequential()
net.add_module('conv1', nn.Conv2d(16, 32, (3, 3), (1, 1)))  # 该卷积层命名为conv1
net.add_module('relu', nn.ReLU())  # 该层命名为relu
net.add_module('flatten', nn.Flatten())  # 该层命名为flatten
net.add_module('linear', nn.Linear(32*62*62, 1))  # 该全连接层命名为linear

该方法使用add_module函数将模块加入到计算图中，每层都有命名。

（3）定义时传入有序字典作为参数

示例如下：

In [15]:
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
('conv1', nn.Conv2d(16, 32, (3, 3), (1, 1))),
('relu', nn.ReLU()),
('flatten', nn.Flatten()),
          ('linear', nn.Linear(32*62*62, 1)),
         ]))

该方法将有序字典作为参数传入，各个神经网络模块作为有序字典的元素。

## 5.3.7 损失函数

PyTorch提供了目前常用的各种损失函数的实现，下面列举一些常用的损失函数示例。

（1）nn.L1Loss

L1Loss即L1损失，也称为平均绝对误差损失（MAE，Mean Absolute Error），其定义如下：

$$ Loss(y)=\frac{1}{m} ∑_{i=1}^{m}|y_{i}-y ̂_{i} |$$

下面给出计算L1Loss的示例：

In [16]:
import torch.nn as nn
import torch
loss = nn.L1Loss()
predict_value = torch.randn(1, 23, requires_grad=True)
target = torch.randn(1, 23)
output = loss(predict_value, target)
print(output)

tensor(1.2751, grad_fn=<L1LossBackward0>)


(2）nn.MSELoss

 MSELoss即平均平方误差（或称均方误差）损失函数，是计算两组数据之间的均方误差，公式如下：

$$ Loss(y)=\frac{1}{m} ∑_{i=1}^{m}(y_{i}-y ̂_{i})^2 $$

nn.MSELoss的使用方法与nn.L1Loss类似，在此不再给出示例。

（3）nn.CrossEntropyLoss

CrossEntropyLoss是进行分类时常用的交叉熵损失函数，可以捕捉不同模型预测效果的差异。对于属于C个类的m个样本，设每个样本i的针对每个类c标签为yic，对于正确标签值为1，错误标签值为0，观测到样本i属于类别c的预测概率为pic，则交叉熵损失函数公式如下：

$$ Loss(y)=\frac{1}{m} ∑_{i=1}^{m}∑_{c=1}^{c}y_{ic}log(p_{ic})$$

下面给出计算CrossEntropyLoss的示例：

In [17]:
torch.manual_seed(0)
p = torch.randn(3, 5, requires_grad=True)  # 得到5个输出结点的值(batch_size=3)
target = torch.empty(3, dtype=torch.long).random_(5)  # 得到每个样本的实际类别标签
print(f'p:{p}\ny:{target}')
loss = torch.nn.CrossEntropyLoss()
output = loss(p, target)  # 计算交叉熵损失
print(f'loss:{output}')

p:tensor([[ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845],
        [-1.3986,  0.4033,  0.8380, -0.7193, -0.4033],
        [-0.5966,  0.1820, -0.8567,  1.1006, -1.0712]], requires_grad=True)
y:tensor([3, 0, 0])
loss:2.272947072982788


示例中可以看出，损失函数的第一个输入参数就是前一层的输出，有时称为Logits输出，可能存在负值或大于1的值，均不能代表概率，第二个输入参数是各个样本的标签数据。损失函数在内部计算时，为了得到样本属于各个类别的概率，通常需要经过softmax函数（或sigmoid函数），然后再可以进行交叉熵损失的计算。